# Linear Regression with Tensorflow using Prechelt Early Stopping
<p class='lead'>
Author: Oliveira, Markos F. B. G.<br />
</p>

# Description

This document is based on Chapter 9 content of Geron's book *Hands-On Machine Learning with Scikit-Learn & TensorFlow*. It covers only the basics of the related topics and it's not meant to be a tutorial. Please, refer to the referenced book and scikit-learn/tensorflow online documentation for specific information.

In paticular, this notebook provides an example of applying linear regression using Tensorflow on moderated-size regression problem. It applies stochastic gradient descent using mini-batches of a given size, two approaches for selecting the next mini-batch exist: *1*: random indices from the training set are selected to be inside the mini-batch using a randomized seed, which is the number of the training iteration. In this case, one epoch, in general, does not cover all the training instances. *2*: a random permutation of the examples is passed and a particular slice of this permutation is used as mini-batch. It's guarateed that all examples are used in any epoch (this approach is the most common).

The algorithm that minimizes the cost function is a plain stochastic gradient descent with mean squared error. An automatic search for good hyperparameters is not implemented, nor ensemble methods are used. However, it implements three stopping criterias presented in *Early Stopping - But When?* paper written by Lutz Prechelt. Finally, the model is retrained using all set of examples.

In [1]:
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
import numpy as np
from datetime import datetime
import time
from sklearn.model_selection import train_test_split

### Fetching and Preprocessing data

In [2]:
#Fetching the data: in the first time it's executed, the data is downloaded to ‘~/scikit_learn_data’ subfolders (this may
#take a while); once, downloaded and executed again, the function just read the available dataset.
housing = fetch_california_housing()
print(fetch_california_housing().keys())
print('Entire dataset shape: ', housing.data.shape)

#Splitting the dataset into training, validation (to early stopping and testing):
X_train_val, X_test, y_train_val, y_test = train_test_split(housing.data, housing.target, random_state=0, test_size=.2)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=0, test_size=.2) #25% of the 80%
#is 20% of the original, so: X_train_ratio = 0.6, X_val_ratio = 0.2, X_test_ratio = 0.2

#Normalizing the data:
scaler = StandardScaler() #instantiates the scaler.
scaler.fit(X_train) #fits the training data, i.e. computes the necessary parameters.
X_train_scaled = scaler.transform(X_train) #Transform the datasets based on parameters evaluated on training set only.
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

#It's important to transform the y_train and y_test 1d-array ((m,) format) to 2d-array ((m,1) format):
y_train = y_train.reshape(-1, 1)
y_val = y_val.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

#Adding the bias feature to each example (necessary for linear regression):
X_train_scaled_plus_bias = np.c_[np.ones((X_train_scaled.shape[0], 1)), X_train_scaled]
X_val_scaled_plus_bias = np.c_[np.ones((X_val_scaled.shape[0], 1)), X_val_scaled]
X_test_scaled_plus_bias = np.c_[np.ones((X_test_scaled.shape[0], 1)), X_test_scaled]
m, n = X_train_scaled_plus_bias.shape

dict_keys(['feature_names', 'DESCR', 'data', 'target'])
Entire dataset shape:  (20640, 8)


### Mini-batch function

In [3]:
def fetch_batch(epoch, batch_index, batch_size, idxs, Xs, ys, approach=2):
    """Returns the mini-batch (X, y) for a training step.

    Parameters
    ----------
    epoch : int
        Number of epoch.
    batch_index : int
        Number of batch inside the epoch.
    batch_size : int
        Batch size.
    idxs : list of int
        Permutation of training set indices.
    approach: int, optional
        Approach used for chosing the mini-batches.
        
    Returns
    -------
    X_batch : np.ndarray
        Mini-batch of inputs.
    y_batch : np.ndarray
        Mini-batch of outputs.
    """
    
    if approach == 1:
        rnd.seed(epoch * n_batches + batch_index)
        indices = rnd.randint(m, size=batch_size)
    elif approach == 2:
        indices = idxs[batch_index * batch_size: (batch_index + 1) * batch_size]
    
    X_batch = Xs[indices]
    y_batch = ys[indices]        
    
    return X_batch, y_batch

### Early-stopping class

Below, the `PrecheltEarlyStopping` class is implemented. It implements stopping criterias presented in *Early Stopping - But When?* paper written by Lutz Prechelt in 1996. Three stopping criteria are presented:

<ol>
  <li>
  Generelization Loss (GL): it measures how much the validation error increased with respect to minimum validation error observed so far (see formula inside the class). This values goes from 0 to +inf. GL = 0 indicates that in the last iteration the validation error was the minimum. GL = 1, indicates that the Eval(t) is 100% (double) above Eopt. See that it doesn't count how many iterations have passed after Eopt; so, if alpha is very big, it can be the case that algorithm never stop (until a upper bound on number of iterations is reached).
  </li>
  <li>
  Generelization Loss / Progress (PQ): progress measures how much was the average error during the last k-length-strip of iterations larger than the minimum training error during the strip. The PQ is defined as the ratio of GL and progress. This criteria is inverselly proportional to progress. Note that in early stage of training, where it's expected that the minimum of the strip is significanty lower than the others (probably previous iterations), the progress is big, which proportionally increases the threshold alpha. In later stages of learning where, probably, all values are common in the strip, the ratio approaches one and the progress approaches zero; this scales down the threshold alpha proportionally. We can see this approach as having a dynamic threshold value for GL according to the oscilations inside the strip (more oscilations means a larger threshold).
  The original approch have a '-1' at the end of 'progress' which makes this metric lower than one if the minimum validation error is below half the final one in the strip. If 'progress' is slower than one, then the aalpha threshold is scaled down proportionally, creating a very fast stopping criteria. The '-1' was then removed so that the 'progress' could only scale up the threshold in early and unstable iterations of training, but never scaling down.
  </li>
  <li>
  UP: it stops the procedure when the generalization error increased in s successive strips (not epochs). It only measures the end of the strips, thus, it tries to capure a general trend error increasing.
  </li>
</ol>

In [4]:
class PrecheltEarlyStopping():
    """Monitor for early stopping in Gradient Boosting for classification.

    The monitor checks the validation loss between each training stage. When
    too many successive stages have increased the loss, the monitor will return
    true, stopping the training early.

    Parameters
    ----------
    X_valid : array-like, shape = [n_samples, n_features]
      Training vectors, where n_samples is the number of samples
      and n_features is the number of features.
    y_valid : array-like, shape = [n_samples]
      Target values (integers in classification, real numbers in
      regression)
      For classification, labels must correspond to classes.
    max_consecutive_decreases : int, optional (default=5)
      Early stopping criteria: when the number of consecutive iterations that
      result in a worse performance on the validation set exceeds this value,
      the training stops.
    
    Returns
    -------
    Save : Boolean
        Save current model.
    Stop : Boolean
        Stop training.
    """

    def __init__(self, approach = 1, gl_alpha = 1, pq_alpha = 2, pq_strip = 10, up_strip = 10, up_s = 3):
        self.approach = approach
        self.gl_alpha = gl_alpha
        self.pq_alpha = pq_alpha
        self.pq_strip = pq_strip
        self.up_strip = up_strip
        self.up_s = up_s
        self.Eopt = float('inf')
        self.strip = []
        
    def early_stopping(self, Eval):
        if Eval <= self.Eopt:
            self.Eopt = Eval
            return True, False #if a new Eopt is found, no criteria will stop training.
        
        if self.approach == 1: #Generalization Loss
            GL = (Eval/self.Eopt) - 1
            if GL > gl_alpha:
                return False, True
            else:
                return False, False        
        elif self.approach == 2: #Generalization Loss / Progress (modified to be slow)
            #Each entry of the strip is an Eval value evaluated when 'early_stopping' is called. If this method is called
            #every N epochs, the strip will have Eval entries spaced by N-1 epochs.
            if len(self.strip) < pq_strip: #In early stages of training where 'early_stopping' function were called N times,
                #where N is less than 'up_strip' (length of the strip), Eval is just appended in strip attribute.
                self.strip.append(Eval)
                return False, False
            else:
                self.strip.pop(0), self.strip.append(Eval)
                Prog = (sum(self.strip)/(len(self.strip)*min(self.strip))) # put -1 in the end of formula to get a very fast
                #stopping criteria
                GL = (Eval/self.Eopt) - 1
                PQ = GL/Prog
                if PQ > pq_alpha:
                    return False, True
                else:
                    return False, False
        elif self.approach == 3: #UP
            if len(self.strip) < up_strip*self.up_s:
            #Before up_strip*s calls, this criteria could note decide if stop or not, it's necessary a minimum of
            #s*strips. 
                self.strip.append(Eval)
                return False, False
            else:
                self.strip.pop(0), self.strip.append(Eval)
                end_strips = [self.strip[i] for i in range(len(self.strip)) if i%self.up_s == 0]
                if sorted(end_strips) == end_strips: #the error only increases
                    return False, True
                else:
                    return False, False

### Linear Regression using TensorFlow

Below, the user can set some running parameters acording to their preference.

In [5]:
#SGD parameters:
n_epochs = 1000
learning_rate = 0.005
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

#Prechelt Early-stopping:
early_stopping_step = 1 #spaced steps in number of epochs that early stopping is checked (it affects PQ and UP approaches).
early_stopping = True #enable early stopping.
stop_learning = True #enable stop learning.
approach = 2 #approach used in early stopping.
gl_alpha = 0.1; pq_alpha = 0.06; pq_strip = 10; up_strip = 5; up_s = 2 #parameters of different sopping criterias.
min_val_error = float("inf") #stores the best error so far.
min_val_error_epoch = 0 #epoch when minimum validation error was found.

#Monitoring:
prt = True #enable print training statistics (in this case MSE).
print_step = 100 # number of epochs to periodically print training statistics.

#Saving/restoring TF model:
restore = False #enable model's restoration.
path_restore = '/tmp/finals/my_model_final.ckpt' #path of original model to be restored.
save_ckpt = False #enable saving training checkpoints.
saver_step = 100 # number of epochs to periodically save model's checkpoint.

#Tensorboard logs:
tb = False #enable log training statistics for tensorboard.
tensorboard_step = 100 #number of epochs to periodically log statistics in tensorboard log files.
root_logdir = "tf_logs" #external folder where all logging stats will be placed (for different session runs).

In [6]:
tf.reset_default_graph() #restoring the default graph.
tf.logging.set_verbosity(tf.logging.WARN) #supress TF logging messages when saving ckpt files.
start_time = time.time()
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
logdir = "{}/run-{}/".format(root_logdir, now) #relative path of tensorboard logs for a particular run (current time
#is used so that each folder has different running stats, comparision between them can be made inside tensorboard).

# TF CONSTRUCTION PHASE

# 1- Creating variables, placeholders and constants.
X = tf.placeholder(tf.float32, shape=(None, n), name="X") #n contains the bias already.
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n, 1], -1.0, 1.0, seed=1), name="theta")

# 2- Creating operations.
y_pred = tf.matmul(X, theta, name="predictions")
with tf.name_scope("loss") as scope: # Grouping related nodes to the same scope in the TF-graph.
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

# 3- Node that initialize the variables.
init = tf.global_variables_initializer()

# 4- Creating the saver.
saver_save = tf.train.Saver(max_to_keep=None) #save all variable values.
if restore:
    saver_restore = tf.train.Saver() #restore all variable values.
    #saver_restore = tf.train.Saver({"weights": theta}) #restore only the old theta variable under the name of 'weights'.

# 5- Tensorboard definitions:
if tb:
    mse_summary = tf.summary.scalar('MSE', mse) #creates a node in the graph that will evaluate the MSE value
    #and write it to a TensorBoard-compatible binary log string called a summary.
    summary_writer = tf.summary.FileWriter(logdir, tf.get_default_graph()) #creates a FileWriter that you will
    #use to write summaries to logfiles in the log directory.


# TF EXECUTION PHASE:

with tf.Session() as sess:
    
    if restore:  #to restore a model the construction phase must be identical than the one used to save it. 
        saver_restore.restore(sess, path_restore)
    else:
        sess.run(init) #initializing the variables.
    
    es = PrecheltEarlyStopping(approach = approach, gl_alpha = gl_alpha, pq_alpha = pq_alpha,
                              pq_strip = pq_strip, up_strip = up_strip, up_s = up_s) #instantiating early-stopping object.
    breaker = False #used to break the epoch loop if the number of iterations without validation improvement increases above
    #the threshold 'n_int'.
    for epoch in range(n_epochs+1): #for each epoch..
        
        idXs = np.random.permutation(range(m)) #creates a random permutation of the instances.
        
        for batch_index in range(n_batches-1): # for each mini-batch.. The '-1' guarantees all batches are equally sized, including
            #the last one, without this the last batch with few examples could be used to update the weights. Hence, a few points
            #(< batch_size) may be out of each epoch.
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size, idXs, X_train_scaled_plus_bias, y_train)
            #Training step:
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            
        if (early_stopping) and (epoch % early_stopping_step == 0):
            Eval = mse.eval(feed_dict={X: X_val_scaled_plus_bias, y: y_val})
            save, stop = es.early_stopping(Eval)
            if save:
                save_path = saver_save.save(sess, "./tmp/best_model-{}.ckpt".format(now))
                min_val_error = Eval
                min_val_error_epoch = epoch #epoch when minimum validation error was found.
            elif stop_learning and stop:
                breaker = True
                break 
                    
        if breaker:
            break 
        if (prt) and (epoch % print_step == 0):
            print("Epoch {}, Training MSE = {:.4}, Validation MSE = {:.4}".format(epoch,
                                                                     mse.eval(feed_dict={X: X_train_scaled_plus_bias, y: y_train}),
                                                                     mse.eval(feed_dict={X: X_val_scaled_plus_bias, y: y_val})))
        if (tb) and (epoch % tensorboard_step == 0):
            summary_str_val = mse_summary.eval(feed_dict={X: X_val_scaled_plus_bias, y: y_cal})   
            summary_str_training = mse_summary.eval(feed_dict={X: X_train_scaled_plus_bias, y: y_train}) # This will output a
            #summary that you can then write to the events file using the file_writer.
            step = epoch * n_batches + batch_index
            summary_writer.add_summary(summary_str_training, step)
            summary_writer.add_summary(summary_str_val, step)
        if (save_ckpt) and (epoch % saver_step == 0):
            save_path = saver_save.save(sess, "./tmp/my_model-{}.ckpt".format(epoch))
          
    #Getting the 'best theta': this will probably not be the best theta over the training set (forget about the test
    #set), because we are updating the weights according the mini batches instead of the full training set error.
    #best_theta = theta.eval()
    #final_error = mse.eval(feed_dict={X: X_test_scaled_plus_bias, y: y_test})
    if not (early_stopping):
        save_path = saver_save.save(sess, "./tmp/best_model-{}.ckpt".format(now))

#Flushing and closing FileWriter if necessary:          
if tb:                            
    summary_writer.flush()
    summary_writer.close() 

if early_stopping:
    print('Epoch {} with minimum validation error {:.4}.'.format(min_val_error_epoch, min_val_error))
best_model_path = "./tmp/best_model-{}.ckpt".format(now)
print("Best model saved as:", best_model_path)
print("Training time: %.8s seconds" % (time.time() - start_time)) 

Epoch 0, Training MSE = 1.378, Validation MSE = 1.524
Epoch 100, Training MSE = 0.5226, Validation MSE = 0.5296
Epoch 200, Training MSE = 0.5276, Validation MSE = 0.5475
Epoch 300, Training MSE = 0.5272, Validation MSE = 0.5465
Epoch 400, Training MSE = 0.5225, Validation MSE = 0.5309
Epoch 160 with minimum validation error 0.5246.
Best model saved as: ./tmp/best_model-20170815205401.ckpt
Training time: 25.57401 seconds


### Testing the best model

In [7]:
saver_restore = tf.train.Saver()
with tf.Session() as sess:
    saver_restore.restore(sess, best_model_path) #restoring the previous model saved.
    best_theta = theta.eval()
    final_error = mse.eval(feed_dict={X: X_test_scaled_plus_bias, y: y_test})

print("\n*-*-*-*-*-*-* Final test results *-*-*-*-*-*-*") 
print("Best theta:\n", best_theta)
print("Final test set error: ", final_error)


*-*-*-*-*-*-* Final test results *-*-*-*-*-*-*
Best theta:
 [[ 2.06496239]
 [ 0.83703643]
 [ 0.11012188]
 [-0.23900716]
 [ 0.28375247]
 [-0.00859088]
 [ 0.00415122]
 [-0.8619861 ]
 [-0.83345944]]
Final test set error:  0.534368


### Building the final model (in the entire dataset available)

For better practical results, it's better to implement the model in the entire dataset available. How much the knowledge of the epoch which minimum validation error could be generalized when training the same model, with the entire data set available? i.e., considering the validation and test set too.
It's common to separate the entire data set (N) available into training (TR), validation (VL) and testing (TS) sets, with proportions 0.6, 0.2 and 0.2 respectively (these values can slighly change).
Considering a training procedure occuring in 60% of the total data, where the minimum validation error occured in epoch Ep. Considering a model with lots of degrees of freedom like neural networks, the early stoppping procedure, avoided overfitting or fitting the particularities of the points available during training that does not encodes intrinsic patterns of the whole population. Increasing the data by a significantly amount, the number of epochs to the overfitting begins will increase, in general. Thus, we could slighly increase the number of epochs during training to capture this phenomena. However,  it's not clear how we should increase the dataset according to individual sets cardinalities. Experiments have to be done to find such factor; in addition it's necessary to compare an approach that only returns the model trained using TR samples, and the one retrained on all dataset.

In [8]:
#Normalizing all the dataset.
housing = fetch_california_housing()
scaler = StandardScaler() #instantiates the scaler.
scaler.fit(housing.data)
X_total = scaler.transform(housing.data)
X_total_plus_bias = np.c_[np.ones((X_total.shape[0], 1)), X_total]
y_total = housing.target.reshape(-1, 1)
m, n = X_total_plus_bias.shape

In [9]:
tf.reset_default_graph() #restoring the default graph.
tf.logging.set_verbosity(tf.logging.WARN) #supress TF logging messages when saving ckpt files.
start_time = time.time()
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
logdir = "{}/run-{}/".format(root_logdir, now)
# TF CONSTRUCTION PHASE

# 1- Creating variables, placeholders and constants.
X = tf.placeholder(tf.float32, shape=(None, n), name="X") #n contains the bias already.
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n, 1], -1.0, 1.0, seed=1), name="theta")

# 2- Creating operations.
y_pred = tf.matmul(X, theta, name="predictions")
with tf.name_scope("loss") as scope: # Grouping related nodes to the same scope in the TF-graph.
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

# 3- Node that initialize the variables.
init = tf.global_variables_initializer()

# 4- Creating the saver.
saver_save = tf.train.Saver(max_to_keep=None) #save all variable values.

factor = 1
n_epochs = int(min_val_error_epoch*factor)
print('New number of epochs: ', n_epochs)

with tf.Session() as sess:
    
    sess.run(init) #initializing the variables.
    
    for epoch in range(n_epochs): #for each epoch..
        
        idXs = np.random.permutation(range(m)) #creates a random permutation of the instances.
        
        for batch_index in range(n_batches-1):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size, idXs, Xs=X_total_plus_bias, ys=y_total)
            #Training step:
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        
    save_path = saver_save.save(sess, "./tmp/final_model-{}.ckpt".format(now))
    print('Final model successifully trained and saved.')

New number of epochs:  160
Final model successifully trained and saved.
